In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0,
                    3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [3]:
def model(t_u, w, b):
    return w * t_u + b

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [5]:
params = torch.tensor([1.0, 0.0], requires_grad=True) # 파라미터 초기화

In [6]:
params.grad is None # pytorch에서 주로 grad의 값은 None

True

In [7]:
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()

In [9]:
params.grad # grad 함수에 누적한다

tensor([4517.2969,   82.6000])

In [12]:
params2 = torch.tensor([2.0, 1.0], requires_grad=True)

In [15]:
loss2 = loss_fn(model(t_u, *params2), t_c)
loss2.backward()
params2.grad

tensor([10502.4941,   188.2000])

In [16]:
# backward 호출은 미분을 말단 노드에 누적하므로 앞서 backward가 호출되었다면 손실이 다시 계산되고 backward가 다시 호출되고 
# 각 말단 노드의 기울기 값이 이전 반복문 수행 시 계산 되었던 기존값에 누적되어 부정확한 기울기 값이 발생
# 각 반복문에서 명시적으로 기울기를 0으로 초기화해야함
if params.grad is not None:
    params.grad.zero_()

In [17]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()

        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        loss.backward()

        with torch.no_grad():
            params -= learning_rate * params.grad

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))

    return params

In [18]:
training_loop(
    n_epochs = 5000,
    learning_rate = 1e-2,
    params = torch.tensor([1.0, 0.0], requires_grad = True),
    t_u = t_un,
    t_c = t_c
)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)